In [1]:
import json, csv

from typing import Dict, List, Tuple
from pathlib import Path
from itertools import islice

In [2]:
# Task 1 & Task 2

# По умолчанию, файл с исходниками лежит в той же папке, что и код, и называется purchase_log.txt / visit_log.csv
_DEFAULT_TEMPLATE_PATH = Path(globals()['_dh'][0]).resolve()
_DEFAULT_SUFFIX_PATHES = {
    '.txt': _DEFAULT_TEMPLATE_PATH / 'purchase_log.txt',
    '.csv': _DEFAULT_TEMPLATE_PATH / 'visit_log.csv'
}


def check_extension(suffix_suggested: str) -> None:
    """Проверка корректности расширения"""

    # Если расширение не .txt и не .csv - выкидываем ошибку
    if suffix_suggested not in _DEFAULT_SUFFIX_PATHES.keys():
        raise ValueError(f"Недопустимый тип файла {suffix_suggested}. Допускаются только {list(_DEFAULT_SUFFIX_PATHES.keys())}'")


def get_source_file_path(source_file_path_suggested: str, suffix_suggested: str) -> Path:
    """Собираем путь к файлу с исходниками и делаем минимальные проверки"""
    
    check_extension(suffix_suggested)
        
    source_file_path = Path(source_file_path_suggested or _DEFAULT_SUFFIX_PATHES[suffix_suggested]).resolve()

    # Если файла нет - выкидываем ошибку
    if not source_file_path.is_file():
        raise FileNotFoundError(f"Файл {source_file_path} не существует")
    
    return source_file_path


def process_purchases_data(source_file_path: str) -> Dict:
    """Собираем словарь из исходных данных в .txt"""

    with open(source_file_path, mode='r', encoding='utf8') as f:
        purchases = {}
        headers = f.readline()
        for line in f:
            text_line = line.rstrip('\n')

            if not text_line:
                continue

            dict_line = json.loads(text_line)

            # Здесь согласно условию задания (значение - строка) предполагаем, что user_id уникальный, иначе столкнемся с потерями данных из-за перезаписи
            purchases[dict_line['user_id']] = dict_line['category']
    
    return purchases


def take(dict_to_iter: Dict, n: int) -> List[Tuple]:
    """Берем первые(если Python >= 3.7) n элементов из итерируемого объекта"""

    return list(islice(dict_to_iter, n))


def pretty_print_purchases(purchases: Dict, num_lines: int = 2) -> None:
    """Печатаем словарь в заданном формате"""

    elems = take(purchases.items(), num_lines)
    
    for user_category in elems:
        print(f"{user_category[0]} '{user_category[1]}'")

In [3]:
# Task 1

purchases_file_path = input("Введите путь к файлу с исходными данными. Пустая строка, если нужно использовать дефолтный: ")

visit_log_file_path_actual = get_source_file_path(purchases_file_path, '.txt')
purchases_dict = process_purchases_data(visit_log_file_path_actual)
pretty_print_purchases(purchases_dict, 2)

1840e0b9d4 'Продукты'
4e4f90fcfb 'Электроника'


In [4]:
# Task 2

def create_funnel_file(visit_log_file_path: Path, funnel_file_path: Path, purchases: Dict) -> None:
    """Процессим visit_log.csv и не отходя пишем в funnel.csv"""

    with open(visit_log_file_path, 'r', encoding='utf8') as visit_log_f, open(funnel_file_path, 'w', encoding='utf8', newline='') as target_file_f:
        visit_log_csv_reader = csv.reader(visit_log_f, delimiter=',')
        funnel_file_csv_writer = csv.writer(target_file_f, delimiter=',')

        # Скипаем заголовки при чтении
        next(visit_log_csv_reader)

        # Пишем заголовки при записи
        headers_to_write = ['user_id', 'source', 'category']
        funnel_file_csv_writer.writerow(headers_to_write)

        for visit_log_row in visit_log_csv_reader:
            # Если user_id есть в словарике покупок, записываем в target_file
            if visit_log_row[0].rstrip() and visit_log_row[0] in purchases:
                funnel_file_csv_writer.writerow([visit_log_row[0], visit_log_row[1], purchases[visit_log_row[0]]]) 


def pretty_print_funnel(funnel_file_path: Path, num_lines: int = 3) -> None:
    """Выводим первые n строк получившегося файла"""

    with open(funnel_file_path, 'r', encoding='utf8') as target_file_f:
        # Создаем ридер
        target_file_csv_reader = csv.reader(target_file_f, delimiter=',')

        # Пишем следующую строчку
        [print(','.join(next(target_file_csv_reader))) for i in range(num_lines)]
    

In [5]:
# Task 2

visit_log_path = input("Введите путь к файлу с исходными данными. Пустая строка, если нужно использовать дефолтный: ")
visit_log_file_path_actual = get_source_file_path(purchases_file_path, '.csv')
funnel_file_path = _DEFAULT_TEMPLATE_PATH / 'funnel.csv'

create_funnel_file(visit_log_file_path_actual, funnel_file_path, purchases_dict)
pretty_print_funnel(funnel_file_path, 3)

user_id,source,category
1840e0b9d4,other,Продукты
4e4f90fcfb,context,Электроника
